# Multi-Objective BO on Aberrations : c1-a1x-a1y

## Installing uv

To start, you'll want to have uv installed:

https://docs.astral.sh/uv/getting-started/installation/


**Next, run this in the terminal, from the parent directory:**

    uv sync


Now you will have the kernel used to run this notebook

---

## Running the Required Servers

You need to run **three servers**, each in its own terminal:

- `central_server.py`
- `AS_server_SimAtomRes.py`
- `Ceos_server_twin.py`

### 1. Activate the Virtual Environment

`uv` should have created a `.venv` directory for you.

**On macOS / Linux:**

    source .venv/bin/activate

**On Windows (likely):**

    source .venv/Scripts/activate

You should now see the environment activated.

---

### 2. Start the Servers

**Terminal 1 — Central Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.protocols.central_server

**Terminal 2 — Atom Resolution Simulation Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.AS_server_SimAtomRes

**Terminal 3 — CEOS Twin Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.Ceos_server_twin

---

You're now ready to run this notebook! 🚀

In [ ]:
import sys
import ast
sys.path.insert(0, '../')
from asyncroscopy.clients.notebook_client import NotebookClient
import matplotlib.pyplot as plt

import pyTEMlib
from pyTEMlib import probe_tools as pt

### Connections:

In [ ]:
# Connect the Client to the central (async) server
tem = NotebookClient.connect(host='localhost',port=9000)

# Tell the central server address of all connected instruments
routing_table= {"AS": ("localhost", 9001),
                "Gatan": ("localhost", 9002),
                "Ceos": ("localhost", 9003),
                "Preacquired_AS": ("localhost", 9004)}
tem.send_command('Central',"set_routing_table", routing_table)

# ConnectionResetError: [Errno 54] Connection reset by peer 
# in terminal, type:
# lsof -i :9000

In [ ]:
# connect to the AutoScript computer and initialize microscope
tem.send_command('AS',command='connect_AS',args={'host':'localhost','port':9001})

In [ ]:
tem.send_command(destination = 'Ceos', command = 'getInfo', args = {})

### Help commands:

In [ ]:
# Now that we're routed to all instruments,
# let's take an inventory of commands available on each instrument
cmds = tem.send_command('AS', 'discover_commands')
print(cmds)

In [ ]:
# These two are working, but should be much better.
tem.send_command('AS', command='get_help', args={'command_name':'connect_AS'})

### Setting the aberrations from known values:

In [41]:
aberrations = pt.get_target_aberrations("Spectra300", 60000)
tem.send_command(destination = 'Ceos', command = 'uploadAberrations', args = aberrations)
pt.print_aberrations(aberrations)

In [ ]:
# at any time, we can view the current aberrations
# this should be implemented in the real ceos server as well
ab = tem.send_command(destination = 'Ceos', command = 'getAberrations', args={})
ab = ast.literal_eval(ab)
pt.print_aberrations(ab)

### Get an image:

simulated with pystemsim inside the AS_server_SimAtomRes (working with the Ceos server)

In [ ]:
image_args = {'scanning_detector':'HAADF',
                'size':512,
                'dwell_time':10e-6}

img = tem.send_command('AS','get_scanned_image', image_args)

plt.imshow(img, cmap="gray")
plt.title("Simulated STEM Image")
plt.show()


### How it's actually working:
![Structure Diagram](../DT_workflow.png)

### try changing an aberration

In [ ]:
tem.send_command(destination = 'Ceos', command = 'correctAberration', args = {"name": 'C10', "value": -6})

In [ ]:
image_args = {'scanning_detector':'HAADF',
                'size':512,
                'dwell_time':10e-6}

img = tem.send_command('AS','get_scanned_image', image_args)

plt.imshow(img, cmap="gray")
plt.title("Simulated STEM Image")
plt.show()

## 2. Lets do BO to tune the parameters

### 2.1 set helper functions

In [ ]:
import matplotlib.pyplot as plt
from scipy import ndimage
import numpy as np

def contrast_rms(im, eps=1e-12):
    m = np.mean(im)
    return np.std(im) / (m + eps)

def get_stem_image_contrast_and_fft(c10, c12a, c12b, plot_diagnostics=False):
    ab['C10'] = c10
    ab['C12a'] = c12a
    ab['C12b'] = c12b

    aberrations = {
        "C10": float(c10),
        "C12a": float(c12a),
        "C12b": float(c12b),
    }


    tem.send_command(
        destination="Ceos",
        command="uploadAberrations",
        args=aberrations,
    )
    # ommand = 'correctAberration' can be also used: Wondering if either is preferred due to speed?
    # tem.send_command(destination = 'Ceos', command = 'correctAberration', args = {"name": 'C10', "value": -6})
    
    
    image_args = {'scanning_detector':'HAADF',
                    'size':512,
                    'dwell_time':10e-6}

    sim_im = tem.send_command('AS','get_scanned_image', image_args)

    ####------> shall we add some noise here?
    
    contrast = contrast_rms(np.array(sim_im))
    
    sim_array = np.array(sim_im, dtype=float)
    sim_array = (sim_array - sim_array.mean()) / sim_array.std()
    
    fft = np.fft.fft2(sim_array)
    fft_shift = np.fft.fftshift(fft)
    power = np.abs(fft_shift)**2
    power_log = np.log1p(power)
    
    center = np.array(power.shape) // 2
    y, x = np.ogrid[:power.shape[0], :power.shape[1]]
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    
    power_log[center[0]-10:center[0]+10, center[1]-10:center[1]+10] = 0
    
    smoothed = ndimage.gaussian_filter(power_log, sigma=2)
    
    threshold = np.percentile(smoothed, 99.5)
    peaks = smoothed > threshold
    
    labeled, num_peaks = ndimage.label(peaks)
    
    if num_peaks == 0:
        fft_score = 0.0
        max_radius = 0
    else:
        peak_distances = []
        for i in range(1, num_peaks + 1):
            peak_coords = np.where(labeled == i)
            peak_y, peak_x = np.mean(peak_coords[0]), np.mean(peak_coords[1])
            distance = np.sqrt((peak_x - center[1])**2 + (peak_y - center[0])**2)
            peak_intensity = smoothed[labeled == i].max()
            if distance > 15:
                peak_distances.append((distance, peak_intensity))
        
        if len(peak_distances) > 0:
            max_radius = max(d[0] for d in peak_distances)
            fft_score = float(max_radius / min(center))
        else:
            max_radius = 0
            fft_score = 0.0
    
    if plot_diagnostics:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        axes[0,0].imshow(sim_im, cmap='gray')
        axes[0,0].set_title('Original Image')
        
        axes[0,1].imshow(power_log, cmap='hot')
        axes[0,1].set_title('FFT Power (log)')
        
        axes[0,2].imshow(smoothed, cmap='hot')
        axes[0,2].set_title('Smoothed FFT')
        
        axes[1,0].imshow(peaks, cmap='gray')
        axes[1,0].set_title(f'Detected Peaks ({num_peaks})')
        
        axes[1,1].imshow(power_log, cmap='hot')
        if num_peaks > 0:
            for i in range(1, num_peaks + 1):
                peak_coords = np.where(labeled == i)
                peak_y, peak_x = np.mean(peak_coords[0]), np.mean(peak_coords[1])
                distance = np.sqrt((peak_x - center[1])**2 + (peak_y - center[0])**2)
                if distance > 15:
                    axes[1,1].plot(peak_x, peak_y, 'rx', markersize=10)
                    if distance == max_radius:
                        axes[1,1].plot(peak_x, peak_y, 'go', markersize=15, fillstyle='none', linewidth=2)
        axes[1,1].plot(center[1], center[0], 'b+', markersize=20)
        axes[1,1].set_title('Peaks Marked (Green=Farthest)')
        
        radial_profile = []
        for rad in range(0, int(min(center))):
            ring_mask = (r >= rad) & (r < rad+1)
            if ring_mask.any():
                radial_profile.append(smoothed[ring_mask].max())
        axes[1,2].plot(radial_profile)
        if max_radius > 0:
            axes[1,2].axvline(max_radius, color='g', linestyle='--', linewidth=2, label=f'Max peak: {max_radius:.1f}px')
        axes[1,2].axhline(threshold, color='r', linestyle='--', label='threshold')
        axes[1,2].set_xlabel('Radius (px)')
        axes[1,2].set_ylabel('Max Power')
        axes[1,2].legend()
        axes[1,2].set_title('Radial Power Profile')
        
        plt.tight_layout()
        plt.suptitle(f'FFT Score: {fft_score:.3f} (max_r={max_radius:.1f}px)', y=1.00, fontsize=14)
        plt.show()
    
    return contrast, fft_score, sim_im




### 2.2 Setup seed data for the BO

In [ ]:
# Parameter ranges based on your exploration
param_ranges = {
    'C10': (-8, 8),    # defocus
    'C12a': (-10, 10), # twofold astigmatism (a)
    'C12b': (-10, 10)  # twofold astigmatism (b)
}

In [ ]:
# Create full grid (coarser for computational efficiency)
n_grid = 7  # 7^3 = 343 points
c10_grid = np.linspace(*param_ranges['C10'], n_grid)
c12a_grid = np.linspace(*param_ranges['C12a'], n_grid)
c12b_grid = np.linspace(*param_ranges['C12b'], n_grid)
C10, C12A, C12B = np.meshgrid(c10_grid, c12a_grid, c12b_grid, indexing='ij')
full_grid = np.stack([C10.flatten(), C12A.flatten(), C12B.flatten()], axis=1)

In [ ]:
# Sample seed points
n_seed = 4
seed_indices = np.random.choice(len(full_grid), n_seed, replace=False)
seed_points = full_grid[seed_indices]

In [ ]:
# ========== QUERY SEED POINTS ==========
print(f"\nQuerying {n_seed} seed points...")
seed_scores = []
seed_images = []

for i, (c10, c12a, c12b) in enumerate(seed_points):
    contrast, fft_score, sim_im = get_stem_image_contrast_and_fft(c10, c12a, c12b, plot_diagnostics=True)
    rewards = np.array((contrast, fft_score))
    seed_scores.append(rewards)
    seed_images.append(sim_im)
    # print(f"Seed {i+1}/{n_seed}, C23a={c23a:.2f}, C23b={c23b:.2f},  C21a={c21a:.2f}, C21b={c21b:.2f}, contrast={contrast:.4f}")

seed_scores = np.array(seed_scores)


In [ ]:
plt.scatter(seed_scores[:, 0], seed_scores[:, 1])

### 2.3 Simple MOBO 

In [ ]:
import torch
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import LogExpectedImprovement
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


from botorch.models import MultiTaskGP
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement, qLogExpectedHypervolumeImprovement 
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning
from botorch.utils.multi_objective import is_non_dominated
from botorch.utils.sampling import sample_simplex
from botorch.utils.transforms import normalize, unnormalize

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# ========== INITIAL SEED POINTS (from previous code) ==========
print(f"Starting with {n_seed} seed points...")
print(f"Best initial contrast: {seed_scores.max():.4f}")

# Convert to tensors
train_X = torch.tensor(seed_points, dtype=torch.float64)
train_Y = torch.tensor(seed_scores, dtype=torch.float64)

# Define bounds for optimization
bounds = torch.tensor([
    [param_ranges['C10'][0], param_ranges['C12a'][0], param_ranges['C12b'][0]],  # lower bounds
    [param_ranges['C10'][1], param_ranges['C12a'][1], param_ranges['C12b'][1]]   # upper bounds
], dtype=torch.float64)



In [ ]:
import torch

# ---- device & dtype ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64

torch.set_default_dtype(dtype)

# move inputs/bounds to device+dtype
train_X = train_X.to(device=device, dtype=dtype)
train_Y = train_Y.to(device=device, dtype=dtype)
bounds  = bounds.to(device=device, dtype=dtype)

n_bo_steps = 50
all_X = train_X.clone()
all_Y = train_Y.clone()
all_images = seed_images.copy()

print("\n" + "="*60)
print("Starting Multi-Objective Bayesian Optimization with EHVI")
print("="*60)

ref_point = train_Y.min(dim=0).values - 0.1 * train_Y.std(dim=0)
print(f"Reference point: {ref_point.detach().cpu().numpy()}")

for step in range(n_bo_steps):
    print(f"\n--- BO Step {step + 1}/{n_bo_steps} ---")
    
    # Train GP (model follows tensor's device/dtype)
    print("Training Multi-Output GP...")
    gp_model = SingleTaskGP(
        all_X, all_Y,
        input_transform=Normalize(d=all_X.shape[-1]).to(device=device, dtype=dtype),
        outcome_transform=Standardize(m=all_Y.shape[-1]).to(device=device, dtype=dtype),
    ).to(device=device, dtype=dtype)
    
    gp_model.likelihood.noise_covar.initialize(noise=0.01)
    mll = ExactMarginalLogLikelihood(gp_model.likelihood, gp_model).to(device=device, dtype=dtype)
    fit_gpytorch_mll(mll)
    
    # EHVI acquisition
    print("Computing Pareto frontier...")
    pareto_mask = is_non_dominated(all_Y)
    pareto_Y = all_Y[pareto_mask]
    print(f"Pareto frontier size: {pareto_Y.shape[0]}")
    
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=pareto_Y)
    EHVI = qLogExpectedHypervolumeImprovement(
        model=gp_model,
        ref_point=ref_point.tolist(),
        partitioning=partitioning,
    )
    
    # Optimize
    print("Optimizing acquisition function...")
    candidate, acq_value = optimize_acqf(
        acq_function=EHVI,
        bounds=bounds,
        q=1,
        num_restarts=10,
        raw_samples=100,
    )
    
    next_X = candidate.detach()
    next_params = next_X.squeeze().detach().cpu().numpy()
    print(f"EHVI value: {acq_value:.6f}")
    
    # Query simulator (returns CPU values)
    print("Querying STEM simulator...")
    objective1, objective2, next_image = get_stem_image_contrast_and_fft(
        next_params[0], next_params[1], next_params[2], plot_diagnostics=True
    )
    next_Y = torch.tensor([[objective1, objective2]], dtype=dtype, device=device)
    
    print(f"Observed objectives: [{objective1:.4f}, {objective2:.4f}]")
    
    # Update tensors on-device
    all_X = torch.cat([all_X, next_X], dim=0)
    all_Y = torch.cat([all_Y, next_Y], dim=0)
    all_images.append(next_image)
    
    new_pareto_mask = is_non_dominated(all_Y)
    if new_pareto_mask[-1]:
        print("✓ NEW PARETO POINT!")


### 2.4 Lets Visualize the Pareto-Frontier and see the MO-BO suggested solutions on it

In [ ]:
# ========== FIND EXTREME AND MID PARETO POINTS ==========
print("\n" + "="*60)
print("Pareto Frontier Analysis")
print("="*60)

final_pareto_mask = is_non_dominated(all_Y)
final_pareto_X = all_X[final_pareto_mask]
final_pareto_Y = all_Y[final_pareto_mask]
pareto_indices = torch.where(final_pareto_mask)[0].cpu().numpy()

print(f"Number of Pareto optimal points: {final_pareto_Y.shape[0]}")

# Find extreme points
extreme_indices = []

# Extreme for Objective 1
max_obj1_idx = torch.argmax(final_pareto_Y[:, 0]).item()
min_obj1_idx = torch.argmin(final_pareto_Y[:, 0]).item()

# Extreme for Objective 2
max_obj2_idx = torch.argmax(final_pareto_Y[:, 1]).item()
min_obj2_idx = torch.argmin(final_pareto_Y[:, 1]).item()

extreme_indices.extend([max_obj1_idx, min_obj1_idx, max_obj2_idx, min_obj2_idx])
extreme_indices = list(set(extreme_indices))  # Remove duplicates

# Find middle point (balanced trade-off)
# Normalize objectives to [0,1] then find point closest to (0.5, 0.5)
normalized_pareto_Y = (final_pareto_Y - final_pareto_Y.min(dim=0).values) / (final_pareto_Y.max(dim=0).values - final_pareto_Y.min(dim=0).values + 1e-8)
distances_to_center = torch.norm(normalized_pareto_Y - 0.5, dim=1)
mid_idx = torch.argmin(distances_to_center).item()

# Combine: extremes + mid
selected_indices = sorted(list(set(extreme_indices + [mid_idx])))

print(f"\nSelected Pareto points for visualization: {len(selected_indices)}")
for idx in selected_indices:
    pareto_idx = pareto_indices[idx]
    params = all_X[pareto_idx].cpu().numpy()
    obj1, obj2 = all_Y[pareto_idx, 0].item(), all_Y[pareto_idx, 1].item()
    
    label = ""
    if idx == max_obj1_idx:
        label += "[MAX Obj1] "
    if idx == min_obj1_idx:
        label += "[MIN Obj1] "
    if idx == max_obj2_idx:
        label += "[MAX Obj2] "
    if idx == min_obj2_idx:
        label += "[MIN Obj2] "
    if idx == mid_idx:
        label += "[MID/Balanced] "
    
    print(f"  {label}")
    print(f"    Obj1={obj1:.4f}, Obj2={obj2:.4f}")
    print(f"    C10={params[0]:.2f}, C12a={params[1]:.2f}, C12b={params[2]:.2f}")

# ========== VISUALIZE ONLY EXTREME + MID PARETO IMAGES ==========
n_selected = len(selected_indices)
n_cols = min(3, n_selected)
n_rows = int(np.ceil(n_selected / n_cols))

fig = plt.figure(figsize=(7*n_cols, 7*n_rows))

for plot_idx, pareto_idx_in_frontier in enumerate(selected_indices):
    ax = fig.add_subplot(n_rows, n_cols, plot_idx + 1)
    
    pareto_idx = pareto_indices[pareto_idx_in_frontier]
    img = all_images[pareto_idx]
    params = all_X[pareto_idx].cpu().numpy()
    obj1, obj2 = all_Y[pareto_idx, 0].item(), all_Y[pareto_idx, 1].item()
    
    # Determine label
    label = ""
    if pareto_idx_in_frontier == max_obj1_idx:
        label = "MAX Obj1"
        color = 'red'
    elif pareto_idx_in_frontier == min_obj1_idx:
        label = "MIN Obj1"
        color = 'blue'
    elif pareto_idx_in_frontier == max_obj2_idx:
        label = "MAX Obj2"
        color = 'green'
    elif pareto_idx_in_frontier == min_obj2_idx:
        label = "MIN Obj2"
        color = 'orange'
    elif pareto_idx_in_frontier == mid_idx:
        label = "BALANCED (Mid)"
        color = 'purple'
    else:
        label = "Extreme"
        color = 'black'
    

    ax.imshow(np.array(img), cmap='gray')
    ax.set_title(
        f'{label}\n'
        f'Obj1={obj1:.4f}, Obj2={obj2:.4f}\n'
        f'C10={params[0]:.1f}, C12a={params[1]:.1f}\n'
        f'C12b={params[2]:.1f}\n',
        fontsize=12,
        fontweight='bold',
        color=color
    )
    ax.axis('off')

plt.tight_layout()
plt.savefig('pareto_extreme_mid_images.png', dpi=150, bbox_inches='tight')
plt.show()

# ========== COMBINED RESULTS PLOT ==========
fig = plt.figure(figsize=(18, 6))

# 1. Objective space
ax1 = plt.subplot(1, 3, 1)
ax1.scatter(all_Y[:, 0].cpu().numpy(), all_Y[:, 1].cpu().numpy(), 
           c='lightblue', s=150, alpha=0.6, edgecolors='gray',
           label='All evaluations')
ax1.scatter(final_pareto_Y[:, 0].cpu().numpy(), final_pareto_Y[:, 1].cpu().numpy(), 
           c='lightcoral', s=200, alpha=0.5, edgecolors='black', 
           linewidths=1, label='Pareto frontier')

# Highlight extreme and mid points
colors = []
labels_legend = []
for idx in selected_indices:
    if idx == max_obj1_idx:
        colors.append('red')
        if 'MAX Obj1' not in labels_legend:
            labels_legend.append('MAX Obj1')
    elif idx == max_obj2_idx:
        colors.append('green')
        if 'MAX Obj2' not in labels_legend:
            labels_legend.append('MAX Obj2')
    elif idx == mid_idx:
        colors.append('purple')
        if 'Balanced' not in labels_legend:
            labels_legend.append('Balanced')
    else:
        colors.append('orange')

for idx, color in zip(selected_indices, colors):
    ax1.scatter(final_pareto_Y[idx, 0].cpu().numpy(), final_pareto_Y[idx, 1].cpu().numpy(),
               c=color, s=400, marker='*', edgecolors='black', linewidths=2, zorder=10)

ax1.set_xlabel('Objective 1 (Contrast)', fontsize=12)
ax1.set_ylabel('Objective 2 (Other Metric)', fontsize=12)
ax1.set_title('Pareto Frontier (★ = Extreme/Mid points)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. BO progress
ax2 = plt.subplot(1, 3, 2)
iterations = range(len(all_Y))
ax2.plot(iterations, all_Y[:, 0].cpu().numpy(), 'o-', label='Objective 1', 
        alpha=0.7, linewidth=2, markersize=8)
ax2.plot(iterations, all_Y[:, 1].cpu().numpy(), 's-', label='Objective 2', 
        alpha=0.7, linewidth=2, markersize=8)
ax2.axvline(len(train_Y)-1, color='red', linestyle='--', 
          label='BO start', linewidth=2)
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Objective Value', fontsize=12)
ax2.set_title('BO Progress', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# 3. Hypervolume
from botorch.utils.multi_objective.hypervolume import Hypervolume

hv_computer = Hypervolume(ref_point=ref_point)
hypervolumes = []
for i in range(len(train_Y), len(all_Y) + 1):
    current_Y = all_Y[:i]
    pareto_mask_i = is_non_dominated(current_Y)
    pareto_Y_i = current_Y[pareto_mask_i]
    hv = hv_computer.compute(pareto_Y_i)
    hypervolumes.append(hv)

ax3 = plt.subplot(1, 3, 3)
ax3.plot(range(len(train_Y), len(all_Y) + 1), hypervolumes, 'o-', 
        linewidth=2, markersize=8, color='purple')
ax3.set_xlabel('Iteration', fontsize=12)
ax3.set_ylabel('Hypervolume', fontsize=12)
ax3.set_title('Hypervolume Improvement', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('multi_objective_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n=== SUMMARY ===")
print(f"Total evaluations: {len(all_Y)}")
print(f"Pareto frontier size: {len(pareto_indices)}")
print(f"Extreme + Mid points shown: {len(selected_indices)}")
print(f"Final hypervolume: {hypervolumes[-1]:.4f}")